In [1]:
from transformers import AutoModel, AutoTokenizer, BertModel
from tqdm import tqdm
import json
import pickle
import re
import random

/home/lc/.conda/envs/pytorch/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
MODEL_PATH = '/home/lc/projects/pretrained_models/chatglm3-6b'
DEVICE = 'cuda:1'                               

In [3]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH, trust_remote_code=True)

In [5]:
query = '''Instruction: 请抽取文本中的三元组并以结构化形式输出
Input: 劳务报酬所得：税法规定的应缴纳个人所得税的各项所得中的一项。是指个人从事设计、装潢、安装、制图、化验、测试、医疗、法律、会计、咨询、讲学、新闻、广播、翻译、审稿、书画、雕刻、影视、录音、录像、演出、表演、广告、展览、技术服务、介绍服务、经纪服务、代办服务以及其他劳务取得的所得。'''

In [5]:
tokenizer.decode(tokenizer.build_single_message('user', '', '你好'), skip_special_tokens=False)

'<|user|> \n 你好'

In [28]:
tokenizer.decode([64796], skip_special_tokens=False)

'<|assistant|>'

In [30]:
tokenizer.decode(tokenizer.build_chat_input(query)["input_ids"][0], skip_special_tokens=False)

'[gMASK]sop<|user|> \n 你好<|assistant|>'

In [4]:
model = AutoModel.from_pretrained(MODEL_PATH, trust_remote_code=True)
model = model.to(DEVICE)
model.eval()

Loading checkpoint shards: 100%|██████████| 7/7 [00:14<00:00,  2.08s/it]


ChatGLMForConditionalGeneration(
  (transformer): ChatGLMModel(
    (embedding): Embedding(
      (word_embeddings): Embedding(65024, 4096)
    )
    (rotary_pos_emb): RotaryEmbedding()
    (encoder): GLMTransformer(
      (layers): ModuleList(
        (0-27): 28 x GLMBlock(
          (input_layernorm): RMSNorm()
          (self_attention): SelfAttention(
            (query_key_value): Linear(in_features=4096, out_features=4608, bias=True)
            (core_attention): CoreAttention(
              (attention_dropout): Dropout(p=0.0, inplace=False)
            )
            (dense): Linear(in_features=4096, out_features=4096, bias=False)
          )
          (post_attention_layernorm): RMSNorm()
          (mlp): MLP(
            (dense_h_to_4h): Linear(in_features=4096, out_features=27392, bias=False)
            (dense_4h_to_h): Linear(in_features=13696, out_features=4096, bias=False)
          )
        )
      )
      (final_layernorm): RMSNorm()
    )
    (output_layer): Linear(in_

In [6]:
query = f'''Task prompt: 请找出冒号左边和右边的词汇之间的关系，是包含、属于、同义、对立、关联还是其他关系

Input: 安家费：费用
Output: 属于

Input: 安家费：安置费
Output: 同义

Input: 爱民活动：送温暖
Output: 包含

Input: 安定团结：社会稳定
Output: 属于

Input: 安定团结：施政方针
Output: 属于

Input: 安全机关：国家安全机关
Output: 同义

Input: 安全：事故
Output: 对立

Input: 安全：保卫
Output: 关联

Input: 安全：安全制度
Output: 包含

Input: 安全：交通安全
Output: 包含

Input: 安全：信息安全
Output: 包含

Input: 安全：国家安全
Output: 包含

Input: 安全：工业安全
Output: 包含

Input: 安全：救助安全
Output: 包含

Input: 安全：旅游安全
Output: 包含

Input: 安全：流通安全
Output: 包含

Input: 安全：燃气智能安全
Output: 包含

Input: 安全：用药安全
Output: 包含

Input: 办公自动化：指挥自动化
Output: 包含

Input: 办公自动化：计算机应用
Output: 属于

Input: 班子建设：领导班子建设
Output: 同义

Input: {left}：{right}
Output: '''

In [6]:
zwws = []

with open('./政务文书.txt', 'r') as f:
    for line in f:
        zwws.append(line.strip())

In [19]:
results = []

In [7]:
left_rights = []

In [8]:
for i in zwws:
    left, right = i.split('：')

    right = right.split('；')

    for r in right:
        left_rights.append((left, r))

In [10]:
len(left_rights)

28088

In [20]:
for left, right in tqdm(left_rights[:5000]):
    query = f'''Task prompt: 请找出冒号左边和右边的词汇之间的关系，是包含、属于、同义、对立、关联还是其他关系

Input: 安家费：费用
Output: 属于

Input: 安家费：安置费
Output: 同义

Input: 爱民活动：送温暖
Output: 包含

Input: 安定团结：社会稳定
Output: 属于

Input: 安定团结：施政方针
Output: 属于

Input: 安全机关：国家安全机关
Output: 同义

Input: 安全：事故
Output: 对立

Input: 安全：保卫
Output: 关联

Input: 安全：安全制度
Output: 包含

Input: 安全：交通安全
Output: 包含

Input: 安全：信息安全
Output: 包含

Input: 安全：国家安全
Output: 包含

Input: 安全：工业安全
Output: 包含

Input: 安全：救助安全
Output: 包含

Input: 安全：旅游安全
Output: 包含

Input: 安全：流通安全
Output: 包含

Input: 安全：燃气智能安全
Output: 包含

Input: 安全：用药安全
Output: 包含

Input: 办公自动化：指挥自动化
Output: 包含

Input: 办公自动化：计算机应用
Output: 属于

Input: 班子建设：领导班子建设
Output: 同义

Input: {left}：{right}
Output: '''
    
    resp, his = model.chat(tokenizer=tokenizer, query=query, temperature=0.3)

    results.append((left, resp.strip(), right))

    

100%|██████████| 5000/5000 [10:34<00:00,  7.88it/s]


In [5]:
with open('results.txt', 'w') as f:
    for s, p, o in results:
        f.write(f'{s}\t{p}\t{o}\n')

In [2]:
import pickle

In [30]:
with open('results.pickle', 'wb') as f:
    pickle.dump(results, f)

In [3]:
with open('results.pickle', 'rb') as f:
    results = pickle.load(f)

In [6]:
resp, his = model.chat(tokenizer=tokenizer, query='晚上睡不着怎么办', do_sample=True, top_k=10, temperature=0.6, max_length=8192)

In [7]:
resp

'晚上睡不着,可以参考下述建议试着改善睡眠:\n1. 尝试放松技巧:深呼吸、冥想、渐进性肌肉松弛等放松技巧可以帮助减轻压力和焦虑,促进睡眠。\n2. 创造舒适的睡眠环境:确保睡眠环境安静、黑暗、凉爽和舒适,例如调整温度、使用耳塞、遮光窗帘等。\n3. 避免刺激性物质:避免在睡前饮用咖啡、茶、可乐、酒精等刺激性物质,这些物质会影响睡眠质量。\n4. 规律的睡眠时间:保持规律的睡眠时间,尽量每天在相同的时间入睡和起床,帮助身体建立健康的睡眠节律。\n5. 避免过度兴奋:避免在睡前过度兴奋的活动,例如使用电子设备、看惊险电影等,让身体放松下来。\n6. 运动:适当的运动可以促进睡眠,但避免在睡前两小时内进行剧烈运动。\n7. 饮食:避免在睡前两小时内进食,特别是高脂肪和高糖分的食物。\n如果以上方法不能改善睡眠问题,建议咨询医生或专业的睡眠医学专家,获取更具体的建议和治疗方案。'

In [27]:
query = '''请抽取下面事件中的事件类型和事件元素，并以json格式返回结果
证券代码：000001 证券简称：平安银行 公告编号：2023-031
优先股代码：140002 优先股简称：平银优 01
平安银行股份有限公司关联交易公告
本公司及董事会全体成员保证信息披露的内容真实、准确、完整，没有虚假记载、误导性陈
述或重大遗漏。
一、关联交易概述
（一）本次关联交易基本情况
平安银行股份有限公司（以下简称“平安银行”或“本行”）第十二届董事会关
联交易控制委员会第六次会议审议通过了《关于与山东黄金矿业股份有限公司关联
交易的议案》，同意与山东黄金矿业股份有限公司（以下简称“山东黄金”） 进行黄
金租赁（套保）业务，交易金额不超过人民币 17.6 亿元。
（二）与上市公司的关联关系
中国平安保险（集团）股份有限公司（以下简称“中国平安”） 为本行的控股股
东，中国平安外部监事刘怀镜先生担任山东黄金独立董事。根据《深圳证券交易所股
票上市规则》相关规定，山东黄金构成本行关联方，本行与上述关联方之间的交易构
成关联交易。
（三）审议表决情况
本行最近一期经审计净资产 4,346.80 亿元，本次关联交易金额不超过人民币 17.6
亿元，本笔交易占本行最近一期经审计净资产 0.40%。本行过去 12 个月及本笔拟与
山东黄金发生的关联交易金额累计达到人民币 36.24 亿元，占本行最近一期经审计净
资产的 0.83%。根据相关监管规定及本行关联交易管理办法，本行与关联方发生的交
易金额占本行最近一期经审计净资产绝对值 0.5%以上的关联交易，应经董事会关联
交易控制委员会审批，并及时披露。
本行第十二届董事会关联交易控制委员会第六次会议审议通过了《关于与山东
黄金矿业股份有限公司关联交易的议案》，无董事回避表决。本行独立董事杨军、艾
春荣、刘峰、吴志攀和郭田勇对前述关联交易进行了事前审核，同意将前述关联交易
提交董事会关联交易控制委员会审议，并就相关事项发表了独立意见。
2
本次关联交易不构成《上市公司重大资产重组管理办法》规定的重大资产重组、
不构成重组上市，不需要经过有关部门批准。
二、关联方基本情况
山东黄金成立于 2001 年 1 月 31 日，注册资本：人民币 4,473,429,525.00 元，注
册地址：济南市历城区经十路 2503 号，法定代表人：李航，统一社会信用代码：
91370000723865016M。经营范围：批准许可范围内的黄金开采、选冶（有效期以许
可证为准）；黄金矿山专用设备、建筑装饰材料（不含国家法律法规限制产品）的生
产、销售。（依法须经批准的项目，经相关部门批准后方可开展经营活动）。截至 2022
年末，山东黄金资产总额 907.22 亿元，负债总额 540.25 亿元，所有者权益 366.96 亿
元，2022 年营业收入 503.06 亿元，利润总额 20.06 亿元，净利润 14.23 亿元。山东
黄金不是失信被执行人。
三、关联交易协议的主要内容
本行第十二届董事会关联交易控制委员会第六次会议审议通过了《关于与山东
黄金矿业股份有限公司关联交易的议案》，同意与山东黄金进行黄金租赁业务合作，
金额不超过人民币 17.6 亿元，费率预计在 2.20%-3.00%，具体以实际报价为准，业
务期限不超过 1 年。
四、关联交易定价政策及定价依据
前述关联交易定价按照商业原则进行，定价以不优于对非关联方同类交易的条
件进行。
五、关联交易目的和影响
本行作为上市的商业银行，本次关联交易为本行的正常业务，且按商业原则协商
订立具体交易条款，定价以不优于对非关联方同类交易的条件进行，不存在损害本行
及股东利益的情形，对本行正常经营活动及财务状况无重大影响。
六、与该关联人累计已发生的关联交易情况
本行过去 12 个月及本次拟与山东黄金发生的关联交易金额累计人民币 36.24 亿
元。
七、独立董事事前认可和独立意见
3
本行独立董事杨军、艾春荣、刘峰、吴志攀和郭田勇对本行《关于与山东黄金矿
业股份有限公司关联交易的议案》进行了事先认可，同意将前述关联交易相关事项提
交董事会关联交易控制委员会审议，并发表独立意见如下：
（一）平安银行关于前述关联交易的表决程序合法合规；
（二）平安银行前述关联交易按商业原则协商订立具体交易条款，定价以不优于
对非关联方同类交易的条件进行，不存在损害本行及股东利益的情形，前述关联交易
也不会影响本行独立性。
八、备查文件
平安银行股份有限公司独立董事认可函和独立意见。
特此公告。
平安银行股份有限公司董事会
2023年7月25日'''
resp, his = model.chat(tokenizer=tokenizer, query=query)

In [28]:
print(resp)

事件类型：关联交易

事件元素：
{
    "证券代码": "000001",
    "证券简称": "平安银行",
    "公告编号": "2023-031",
    "关联交易概述": {
        "关联交易基本情况": {
            "交易金额": "人民币 17.6 亿元",
            "交易占净资产比例": "0.40%",
            "过去 12 个月及本次关联交易金额累计达到人民币 36.24 亿元，占最近一期经审计净资产的 0.83%"
        },
        "与上市公司的关联关系": {
            "中国平安": {
                "关联关系": "控股股东",
                "独立董事": {
                    "刘怀镜": {
                        "关联关系": "外部监事",
                        "角色": "独立董事"
                    }
                }
            },
            "山东黄金": {
                "关联关系": "关联方",
                "业务类型": "黄金租赁（套保）业务"
            }
        },
        "审议表决情况": {
            "董事会关联交易控制委员会": {
                "表决程序": "合法合规",
                "表决结果": "无董事回避表决",
                "独立董事": {
                    "杨军、艾春荣、刘峰、吴志攀和郭田勇": {
                        "审核意见": "同意将前述关联交易提交董事会关联交易控制委员会审议，并就相关事项发表了独立意见",
                        "回避表决": "无"
                    }
                }
            }


In [16]:
print(resp)

事件类型：关联交易

事件元素：
{
    "证券代码": "000001",
    "证券简称": "平安银行",
    "公告编号": "2023-031",
    "关联交易概述": {
        "关联交易基本情况": {
            "交易金额": "人民币 17.6 亿元",
            "交易占净资产比例": "0.40%",
            "关联交易金额累计": "人民币 36.24 亿元",
            "关联交易占净资产比例": "0.83%"
        },
        "与上市公司的关联关系": {
            "关联方": "中国平安保险（集团）股份有限公司",
            "关联关系": "控股股东",
            "关联董事": "刘怀镜先生"
        },
        "审议表决情况": {
            "表决会议": "第十二届董事会关联交易控制委员会第六次会议",
            "表决结果": "无董事回避表决",
            "独立董事审核": {
                "审核意见": "同意将前述关联交易提交董事会关联交易控制委员会审议",
                "独立意见": "本行与关联方发生的交易构成为关联交易。本次关联交易不构成《上市公司重大资产重组管理办法》规定的重大资产重组、不构成重组上市，不需要经过有关部门批准。"
            }
        }
    },
    "关联方基本情况": {
        "公司名称": "山东黄金矿业股份有限公司",
        "注册资本": "人民币 4,473,429,525.00 元",
        "注册地址": "济南市历城区经十路 2503 号",
        "法定代表人": "李航",
        "统一社会信用代码": "91370000723865016M",
        "经营范围": "黄金开采、选冶、矿山专用设备、建筑装饰材料（不含国家法律法规限制产品）的生产、销售。",
        "截至 2022 年末": {
           

In [36]:
data = '''{
    "证券代码": "000001",
    "证券简称": "平安银行",
    "公告编号": "2023-031",
    "关联交易概述": {
        "关联交易基本情况": {
            "交易金额": "人民币 17.6 亿元",
            "交易占净资产比例": "0.40%"
        },
        "与上市公司的关联关系": {
            "中国平安": {
                "关联关系": "控股股东",
                "独立董事": {
                    "刘怀镜": {
                        "关联关系": "外部监事",
                        "角色": "独立董事"
                    }
                }
            },
            "山东黄金": {
                "关联关系": "关联方",
                "业务类型": "黄金租赁（套保）业务"
            }
        },
        "审议表决情况": {
            "董事会关联交易控制委员会": {
                "表决程序": "合法合规",
                "表决结果": "无董事回避表决",
                "独立董事": {
                    "杨军、艾春荣、刘峰、吴志攀和郭田勇": {
                        "审核意见": "同意将前述关联交易提交董事会关联交易控制委员会审议，并就相关事项发表了独立意见",
                        "回避表决": "无"
                    }
                }
            }
        }
    },
    "关联方基本情况": {
        "山东黄金": {
            "基本情况": {
                "成立时间": "2001 年 1 月 31 日",
                "注册资本": "人民币 4,473,429,525.00 元",
                "注册地址": "济南市历城区经十路 2503 号",
                "法定代表人": "李航",
                "统一社会信用代码": "91370000723865016M",
                "经营范围": {
                    "黄金开采、选冶": {
                        "有效期以许可证为准"
                    },
                    "黄金矿山专用设备、建筑装饰材料（不含国家法律法规限制产品）的生产、销售"
                },
                "资产总额": "907.22 亿元",
                "负债总额": "540.25 亿元",
                "所有者权益": "366.96 亿元",
                "2022 年营业收入": "503.06 亿元",
                "利润总额": "20.06 亿元",
                "净利润": "14.23 亿元"
            },
            "关联关系": {
                "山东黄金不是失信被执行人"
            }
        }
    },
    "关联交易协议的主要内容": {
        "黄金租赁业务合作": {
            "合作金额": "人民币 17.6 亿元",
            "费率": "2.20%-3.00%",
            "业务期限": "1 年",
            "定价以不优于对非关联方同类交易的条件下进行"
        }
    },
    "关联交易定价政策及定价依据": {
        "按照商业原则进行定价"
    },
    "关联交易目的和影响": {
        "本行作为上市的商业银行，本次关联交易为本行的正常业务，且按商业原则协商订立具体交易条款，定价以不优于对非关联方同类交易的条件进行，不存在损害本行及股东利益的情形，对本行正常经营活动及财务状况无重大影响"
    },
    "与该关联人累计已发生的关联交易情况": {
        "过去 12 个月及本次关联交易金额累计人民币 36.24 亿元"
    },
    "独立董事事前认可和独立意见": {
        "平安银行关于前述关联交易的表决程序合法合规；平安银行前述关联交易按商业原则协商订立具体交易条款，定价以不优于对非关联方同类交易的条件进行，不存在损害本行及股东利益的情形，前述关联交易也不会影响本行独立性"
    },
    "备查文件": {
        "平安银行股份有限公司独立董事认可函和独立意见"
    }
}'''

In [20]:
json.dumps(json.loads(data), ensure_ascii=False, indent=4)

'{\n    "证券代码": "000001",\n    "证券简称": "平安银行",\n    "公告编号": "2023-031",\n    "关联交易概述": {\n        "关联交易基本情况": {\n            "交易金额": "人民币 17.6 亿元",\n            "交易占净资产比例": "0.40%",\n            "关联交易金额累计": "人民币 36.24 亿元",\n            "占净资产比例": "0.83%"\n        },\n        "与上市公司的关联关系": {\n            "关联方": "中国平安保险（集团）股份有限公司",\n            "关联关系": "控股股东",\n            "关联方角色": "独立董事"\n        },\n        "审议表决情况": {\n            "表决会议": "第十二届董事会关联交易控制委员会第六次会议",\n            "表决结果": "通过",\n            "回避表决": "无"\n        }\n    },\n    "关联方基本情况": {\n        "公司名称": "山东黄金矿业股份有限公司",\n        "注册资本": "人民币 4,473,429,525.00 元",\n        "注册地址": "济南市历城区经十路 2503 号",\n        "法定代表人": "李航",\n        "统一社会信用代码": "91370000723865016M",\n        "经营范围": "黄金开采、选冶、设备生产、销售",\n        "资产总额": "907.22 亿元",\n        "负债总额": "540.25 亿元",\n        "所有者权益": "366.96 元",\n        "2022 年营业收入": "503.06 亿元",\n        "利润总额": "20.06 亿元",\n        "净利润": "14.23 亿元",\n        "是否失信被执行人": "不是"\n    },\n    "关联交易

In [37]:
eval(data)

SyntaxError: ':' expected after dictionary key (<string>, line 50)

In [34]:
json.loads(data)

JSONDecodeError: Expecting ':' delimiter: line 10 column 9 (char 249)

In [33]:
print(data)

{
    "证券代码": "000001",
    "证券简称": "平安银行",
    "公告编号": "2023-031",
    "关联交易概述": {
        "关联交易基本情况": {
            "交易金额": "人民币 17.6 亿元",
            "交易占净资产比例": "0.40%",
            "过去 12 个月及本次关联交易金额累计达到人民币 36.24 亿元，占最近一期经审计净资产的 0.83%"
        },
        "与上市公司的关联关系": {
            "中国平安": {
                "关联关系": "控股股东",
                "独立董事": {
                    "刘怀镜": {
                        "关联关系": "外部监事",
                        "角色": "独立董事"
                    }
                }
            },
            "山东黄金": {
                "关联关系": "关联方",
                "业务类型": "黄金租赁（套保）业务"
            }
        },
        "审议表决情况": {
            "董事会关联交易控制委员会": {
                "表决程序": "合法合规",
                "表决结果": "无董事回避表决",
                "独立董事": {
                    "杨军、艾春荣、刘峰、吴志攀和郭田勇": {
                        "审核意见": "同意将前述关联交易提交董事会关联交易控制委员会审议，并就相关事项发表了独立意见",
                        "回避表决": "无"
                    }
                }
            }
        }
    },


In [38]:
his

[{'role': 'user', 'content': '晚上睡不着怎么办'},
 {'role': 'assistant',
  'metadata': '',
  'content': '晚上睡不着,可以参考下述建议试着改善睡眠:\n1. 建立规律的睡眠时间:尽可能每天在相同的时间入睡和起床,包括周末。\n2. 放松身心:睡前做些放松的活动,如泡澡、冥想、听轻柔的音乐等。避免在睡前使用电子设备或进行刺激性活动。\n3. 避免过度进食:晚餐过量或饮食过于刺激的食物可能会影响睡眠。\n4. 创造舒适的睡眠环境:保持卧室的温度适宜、安静和暗淡,可能需要使用耳塞或白噪音等工具。\n5. 运动:适量的运动可以帮助改善睡眠,但避免在睡前进行过于剧烈的活动。\n6. 限制咖啡因和酒精:咖啡因和酒精可能会影响睡眠质量,尽可能避免在睡前饮用。\n7. 放松技巧:尝试一些放松技巧,如深呼吸、渐进性肌肉松弛或瑜伽等。\n8. 如果您长时间睡不着,建议咨询医生或专业心理医生的意见。\n改善睡眠需要耐心和时间,不要过于焦虑或强求,逐步调整生活方式和睡眠习惯即可。'},
 {'role': 'user', 'content': '放松不了怎么办'},
 {'role': 'assistant',
  'metadata': '',
  'content': '放松不了,可以参考下述建议:\n1. 改变思维方式:有时候,过度思考和焦虑会让人难以放松。试着转移注意力,将注意力放在呼吸或身体的感觉上,或者尝试一些冥想或深度放松技巧。\n2. 调整环境:创造一个安静、舒适和黑暗的环境,可能需要关闭窗户或拉上窗帘,使用耳塞或白噪音等工具。\n3. 放松身体:通过渐进性肌肉松弛或深度呼吸等技巧来放松身体。先将注意力放在脚上,然后逐渐向上移动至头部。\n4. 放松心情:尝试一些心理放松技巧,如深呼吸、冥想、想象一个开心的场景等。避免过度思考和焦虑。\n5. 适度使用电子设备:有时候,使用电子设备可以让人感到放松,但过度使用可能会影响睡眠。尝试在睡前限制使用电子设备的时间和频率。\n6. 寻求帮助:如果长时间无法放松,建议咨询专业心理医生的意见。他们可以提供更深入的建议和技巧,帮助缓解压力和焦虑。\n放松是一个需要练习和改进的过程,不要过于强求,逐步调整生活方式和习惯即可。'}]

In [25]:
encoded = tokenizer.build_chat_input('如何改变思维方式', history=[])

In [26]:
print(tokenizer.decode(encoded["input_ids"][0], skip_special_tokens=False))

[gMASK]sop<|user|> 
 如何改变思维方式<|assistant|>


In [40]:
encoded

{'input_ids': tensor([[64790, 64792, 64795, 30910,    13, 30910, 32820, 54266, 35153, 64796,
         30910,    13, 30910, 32820, 54266, 30932, 31628, 33214, 54578, 55340,
         32108, 45318, 32537, 34192, 30954,    13, 30939, 30930, 30910, 31931,
         33700, 54530, 34192, 31643, 30954, 37159, 32096, 54534, 33360, 32631,
         45763, 54542, 39360, 30932, 31779, 35042, 31155,    13, 30943, 30930,
         30910, 34794, 34532, 30954, 43612, 54725, 54727, 34794, 37498, 30932,
         54627, 56049, 57715, 31201, 51956, 31201, 55208, 55215, 56059, 41512,
         54609, 31155, 32317, 54534, 43612, 31695, 32157, 32077, 54746, 31636,
         33464, 54642, 31651, 31155,    13, 30966, 30930, 30910, 32317, 35700,
         44768, 30954, 40028, 50182, 54746, 33296, 34583, 33464, 36899, 32934,
         31731, 34192, 31155,    13, 30972, 30930, 30910, 32320, 47108, 34192,
         31747, 30954, 31983, 40210, 54530, 33576, 39127, 31201, 36055, 54542,
         55935, 55881, 30932, 31667, 3

# 人社

In [41]:
from py2neo import Graph, Node
from tqdm import tqdm
import pandas as pd
import numpy as np

In [10]:
g = Graph("http://127.0.0.1:7474", name="neo4j", password="15623792690")

In [62]:
def create_node(label, nodes):
    print(f'creating {label} nodes')
    for node in tqdm(nodes):
        if isinstance(label, list):
            node = Node(*label, **node)
        else:
            node = Node(label, **node)
        g.create(node)
        # print('created node', label, node_name)


def create_relationship(start_nodeType, end_nodeType, s_attr, e_attr, edges, rel_type):
    # print(f'creating {rel_type} relationships')
    for edge in tqdm(edges):
        start = edge[0]
        end = edge[1]

        if len(edge) == 2:
            query = f'match(p:{start_nodeType}),(q:{end_nodeType}) where p.{s_attr}="{start}" and q.{e_attr}="{end}" create (p)-[rel:{rel_type}]->(q)'
        else:
            attribute = edge[2]
            attrs = '{'
            for k, v in attribute.items():
                attrs += f'{k}: "{v}",'
            attrs = attrs[:-1] + '}'
            query = f'match(p:{start_nodeType}),(q:{end_nodeType}) where p.{s_attr}="{start}" and q.{e_attr}="{end}" create (p)-[rel:{rel_type}{attrs}]->(q)'
        try:
            g.run(query)
        except Exception as e:
            print(e)

In [21]:
rs_nodes = [{'rs_Id': 'rs1', 'name': '人社'}]

In [26]:
create_node('人社', rs_nodes)

creating 人社 nodes


100%|██████████| 1/1 [00:00<00:00, 16.03it/s]


In [1]:
import pandas as pd

In [42]:
df = pd.read_csv('./人社/人社部内设机构.csv')

In [45]:
df['name'].tolist()

['办公厅',
 '政策研究司',
 '法规司',
 '规划财务司',
 '就业促进司',
 '人力资源流动管理司',
 '职业能力建设司',
 '专业技术人员管理司',
 '事业单位人事管理司',
 '农民工工作司',
 '劳动关系司',
 '工资福利司',
 '养老保险司',
 '失业保险司',
 '工伤保险司',
 '农村社会保险司',
 '社会保险基金监管局',
 '调解仲裁管理司',
 '劳动保障监察局',
 '国家表彰奖励办公室',
 '国际合作司（港澳台办公室）',
 '人事司']

In [111]:
responsibilities = [
    "方针政策贯彻落实", "教育、管理、监督、服务制度", "政治思想建设", "党组织建设", "离退休干部宣传工作", "文化体育和老年教育活动",
    "精准服务", "医疗保健和健康管理", "丧葬和善后事宜", "走访慰问工作", "经费预决算", "资产管理", "部领导交办事项"
]


In [112]:
with open('./人社/人社部内设机构-机构职责.txt', 'a+') as f:
    for r in responsibilities:
        f.write(f'离退休干部局\t{r}\n')

In [49]:
rs_organization_nodes = []

num = 1
for i in df['name'].tolist():
    rs_organization_nodes.append({
        'rs_organization_Id': f'rs_organization{num}',
        'name': i
    })
    num += 1

In [50]:
rs_organization_nodes

[{'rs_organization_Id': 'rs_organization1', 'name': '办公厅'},
 {'rs_organization_Id': 'rs_organization2', 'name': '政策研究司'},
 {'rs_organization_Id': 'rs_organization3', 'name': '法规司'},
 {'rs_organization_Id': 'rs_organization4', 'name': '规划财务司'},
 {'rs_organization_Id': 'rs_organization5', 'name': '就业促进司'},
 {'rs_organization_Id': 'rs_organization6', 'name': '人力资源流动管理司'},
 {'rs_organization_Id': 'rs_organization7', 'name': '职业能力建设司'},
 {'rs_organization_Id': 'rs_organization8', 'name': '专业技术人员管理司'},
 {'rs_organization_Id': 'rs_organization9', 'name': '事业单位人事管理司'},
 {'rs_organization_Id': 'rs_organization10', 'name': '农民工工作司'},
 {'rs_organization_Id': 'rs_organization11', 'name': '劳动关系司'},
 {'rs_organization_Id': 'rs_organization12', 'name': '工资福利司'},
 {'rs_organization_Id': 'rs_organization13', 'name': '养老保险司'},
 {'rs_organization_Id': 'rs_organization14', 'name': '失业保险司'},
 {'rs_organization_Id': 'rs_organization15', 'name': '工伤保险司'},
 {'rs_organization_Id': 'rs_organization16', 'name': 

In [52]:
create_node('人社部内设机构', rs_organization_nodes)

creating 人社部内设机构 nodes


100%|██████████| 22/22 [00:00<00:00, 110.33it/s]


In [53]:
rels_rs_rs_organization = []

for i in df['name'].tolist():
    rels_rs_rs_organization.append(['人社', i])

In [64]:
create_relationship('人社', '人社部内设机构', 'name', 'name', rels_rs_rs_organization, '人社部内设机构')

100%|██████████| 22/22 [00:00<00:00, 104.75it/s]


In [ ]:
rs_system_set = set()

with open('./人社/人社核心要素-人社核心要素关键点.txt', 'r') as f:
    for line in f:
        _, name = line.split('\t')

        rs_system_set.add(name.strip())

rs_system_nodes = []

num = 1
for i in rs_system_set:
    rs_system_nodes.append({'rs_system_Id': f'rs_system{num}', 'name': i})
    num += 1

In [120]:
organization_duty_set = set()

with open('./人社/人社部内设机构-机构职责.txt', 'r') as f:
    for line in f:
        _, name = line.split('\t')

        organization_duty_set.add(name.strip())

organization_duty_nodes = []

num = 1
for i in organization_duty_set:
    organization_duty_nodes.append({
        'organization_duty_Id': f'organization_duty{num}',
        'name': i
    })
    num += 1

In [123]:
create_node('机构职责', organization_duty_nodes)

creating 机构职责 nodes


100%|██████████| 255/255 [00:02<00:00, 118.64it/s]


In [124]:
rels_organization_organization_duty = []

with open('./人社/人社部内设机构-机构职责.txt', 'r') as f:
    for line in f:
        a, b = line.split('\t')
        relation = [a, b.strip()]
        if relation not in rels_organization_organization_duty:
            rels_organization_organization_duty.append(relation)

In [126]:
create_relationship('人社部内设机构', '机构职责', 'name', 'name', rels_organization_organization_duty, '职责')

100%|██████████| 313/313 [00:02<00:00, 113.42it/s]


In [59]:
rels_rs_rs_system = []

for i in rs_system_set:
    rels_rs_rs_system.append(['人社', i])

In [63]:
create_relationship('人社', '人社体系', 'name', 'name', rels_rs_rs_system, '人社体系')

100%|██████████| 37/37 [00:00<00:00, 110.80it/s]


In [15]:
system_management_set = set()

with open('./人社/人社核心要素关键点-人社核心要素细化关键点.txt', 'r') as f:
    for line in f:
        _, name = line.split('\t')

        system_management_set.add(name.strip())

system_management_nodes = []

num = 1
for i in system_management_set:
    system_management_nodes.append({
        'system_management_Id': f'system_management{num}',
        'name': i
    })
    num += 1

In [23]:
len(system_management_set)

344

In [19]:
create_node('体系管理', system_management_nodes)

creating 体系管理 nodes


100%|██████████| 344/344 [00:02<00:00, 124.33it/s]


In [17]:
system_management_nodes

[{'system_management_Id': 'system_management1', 'name': '养老保险转移'},
 {'system_management_Id': 'system_management2', 'name': '人力资源招聘与配置'},
 {'system_management_Id': 'system_management3', 'name': '人力资源使用评价'},
 {'system_management_Id': 'system_management4', 'name': '生育保险政策'},
 {'system_management_Id': 'system_management5', 'name': '社会福利需求'},
 {'system_management_Id': 'system_management6', 'name': '异地就医结算服务'},
 {'system_management_Id': 'system_management7', 'name': '优抚标准'},
 {'system_management_Id': 'system_management8', 'name': '人才培养基地'},
 {'system_management_Id': 'system_management9', 'name': '人力资源培训与教育'},
 {'system_management_Id': 'system_management10', 'name': '人力资源激励与约束'},
 {'system_management_Id': 'system_management11', 'name': '社会救助的管理'},
 {'system_management_Id': 'system_management12', 'name': '激励方式'},
 {'system_management_Id': 'system_management13', 'name': '医疗保险审核'},
 {'system_management_Id': 'system_management14', 'name': '公益事业发展规划'},
 {'system_management_Id': 'system_management1

In [30]:
system_element_set = set()

with open('./人社/人社核心要素细化关键点-人社核心要素细化关键点核心词.txt', 'r') as f:
    for line in f:
        _, name = line.split('\t')

        system_element_set.add(name.strip())

system_element_nodes = []

num = 1
for i in system_element_set:
    system_element_nodes.append({
        'system_element_Id': f'system_element{num}',
        'name': i
    })
    num += 1

In [28]:
len(system_element_nodes)

2070

In [32]:
create_node('体系要素', system_element_nodes)

creating 体系要素 nodes


100%|██████████| 2070/2070 [00:20<00:00, 101.68it/s]


In [11]:
create_node('人社核心要素细化关键点核心词', corerskeymoreword_nodes)

creating 人社核心要素细化关键点核心词 nodes


100%|██████████| 2070/2070 [00:19<00:00, 106.31it/s]


In [12]:
rs_keys_nodes = []

num = 1
for key in rs_keys:
    rs_keys_nodes.append({'corerskey_Id': f'corerskey{num}', 'name': key})
    num += 1

In [18]:
create_node('人社核心要素关键点', rs_keys_nodes)

creating 人社核心要素关键点 nodes


100%|██████████| 37/37 [00:00<00:00, 71.31it/s]


In [2]:
rels_rscore_rscorekey = []

In [24]:
rels_rs_system_system_management = []

with open('./人社/人社核心要素关键点-人社核心要素细化关键点.txt', 'r') as f:
    for line in f:
        # rs_keys.append(line.split('\t')[1].strip())
        a, b = line.split('\t')
        relation = [a, b.strip()]
        if relation not in rels_rs_system_system_management:
            rels_rs_system_system_management.append([a, b.strip()])

In [26]:
create_relationship('人社体系', '体系管理', 'name', 'name', rels_rs_system_system_management, '包括')

100%|██████████| 354/354 [00:04<00:00, 85.85it/s]


In [34]:
rels_rs_system_system_element = []

with open('./人社/人社核心要素细化关键点-人社核心要素细化关键点核心词.txt', 'r') as f:
    for line in f:
        # rs_keys.append(line.split('\t')[1].strip())
        a, b = line.split('\t')
        relation = [a, b.strip()]
        if relation not in rels_rs_system_system_element:
            rels_rs_system_system_element.append([a, b.strip()])

In [40]:
create_relationship('体系管理', '体系要素', 'name', 'name', rels_rs_system_system_element, '')

100%|██████████| 4547/4547 [00:44<00:00, 102.72it/s]


In [ ]:
rels_rscorekey_rscore = []

with open('./人社/人社核心要素关键点-人社核心要素细化关键点.txt', 'r') as f:
    for line in f:
        # rs_keys.append(line.split('\t')[1].strip())
        a, b = line.split('\t')
        rels_rscore_rscorekey.append([a, b.strip()])


In [50]:
from tqdm import tqdm

In [9]:
with open('人社核心要素关键点-人社核心要素细化关键点.txt', 'a+') as f:
    for key in tqdm(rs_keys):
        query = f'''Input: 中国人社体系中的人力资源战略规划主要包含哪些内容，请将关键词放在一个Python的列表中作为答案返回
        Output: ["人力资源市场分析", "人力资源需求预测", "人力资源供给与需求平衡", "人力资源政策与法规", "人才培养与发展", "人才引进与使用", "人力资源信息与监测", "社会保障体系", "劳动争议处理", "企业人力资源管理"]

        Input: 中国人社体系中的人才引进主要包含哪些内容，请将关键词放在一个Python的列表中作为答案返回
        Output: ["人才引进政策", "引进人才的类别", "引进人才的程序", "引进人才的标准", "引进人才的待遇", "人才引进工作流程", "引进人才的渠道", "引进人才的管理", "人才引进服务"]

        Input: 中国人社体系中的{key}主要包含哪些内容，请将关键词放在一个Python的列表中作为答案返回
        Output: '''

        resp, his = model.chat(tokenizer=tokenizer, query=query)

        try:
            result = eval(resp)
            for r in result:
                f.write(f'{key}\t{r}\n')
        except Exception as e:
            print(key)


100%|██████████| 37/37 [00:46<00:00,  1.25s/it]


In [12]:
with open('./人社/人社核心要素细化关键点-人社核心要素细化关键点核心词.txt', 'a+') as f:
    for key in tqdm(['最低生活保障的争议处理']):
        query = f'''Input: 中国人社体系中的劳动保障监察对象主要包含哪些内容，请将关键词放在一个Python的列表中作为答案返回
        Output: ["劳动合同", "工资支付", "社会保险", "劳动保险", "工作时间", "休息休假", "劳动标准", "劳动保护", "女职工权益", "未成年工保护", "职业危害防护", "劳动争议", "劳动安全", "失业保险", "工伤保险"]

        Input: 中国人社体系中的劳动力市场政策主要包含哪些内容，请将关键词放在一个Python的列表中作为答案返回
        Output: ["就业促进", "职业培训", "失业保险", "劳动力流动", "就业服务", "灵活就业", "技能提升", "农民工就业", "人才引进", "社会保障", "劳动合同", "职业安全", "职业健康", "工资保障", "劳动标准", "劳动关系"]

        Input: 中国人社体系中的{key}主要包含哪些内容，请将关键词放在一个Python的列表中作为答案返回
        Output: '''

        resp, his = model.chat(tokenizer=tokenizer, query=query)

        try:
            result = eval(resp)
            for r in result:
                f.write(f'{key}\t{r}\n')
        except Exception as e:
            print(key)

100%|██████████| 1/1 [00:01<00:00,  1.02s/it]

最低生活保障的争议处理


In [1]:
resp = '["最低生活保障", "争议处理", "社会救助", "社会保障", "困难群体", "贫困地区", "生活保障线", "补偿标准", "权益保障", "法律制度", "调解仲裁", "司法诉讼", "社会监督", "政策衔接"]'

In [3]:
key = '最低生活保障的争议处理'

In [33]:
resp

'["最低生活保障", "争议处理", "社会救助", "社会保障", "困难群体", "贫困地区", "生活保障线", "补偿标准", "权益保障", "法律制度", "调解仲裁", "司法诉讼", "社会监督", "政策衔接"]'

In [2]:
cultural_security_policy = [
    "文化遗产保护", "文化传承传播", "文化产业支持", "文化消费促进", "文化公共服务", "非物质文化遗产保护", "文化多样性保护",
    "文化产权保护", "文化创意产业扶持", "文化教育支持", "文化产业政策", "文化艺术交流", "文化产业国际合作", "文化创新保障",
    "文化产业法规"
]


In [3]:
key = '文化保障政策'
with open('./人社/人社核心要素细化关键点-人社核心要素细化关键点核心词.txt', 'a+') as f:
    # result = eval(resp)
    for r in cultural_security_policy:
        f.write(f'{key}\t{r}\n')

In [17]:
query = '''Input: 中国人社体系中的劳动保障监察对象主要包含哪些内容，请将关键词放在一个Python的列表中作为答案返回
Output: ["劳动合同", "工资支付", "社会保险", "劳动保险", "工作时间", "休息休假", "劳动标准", "劳动保护", "女职工权益", "未成年工保护", "职业危害防护", "劳动争议", "劳动安全", "失业保险", "工伤保险"]

Input: 中国人社体系中的劳动力市场政策主要包含哪些内容，请将关键词放在一个Python的列表中作为答案返回
Output: ["就业促进", "职业培训", "失业保险", "劳动力流动", "就业服务", "灵活就业", "技能提升", "农民工就业", "人才引进", "社会保障", "劳动合同", "职业安全", "职业健康", "工资保障", "劳动标准", "劳动关系"]

Input: 中国人社体系中的文化保障政策主要包含哪些内容，请将关键词放在一个Python的列表中作为答案返回
Output: '''
resp, his = model.chat(tokenizer=tokenizer, query=query)

In [18]:
eval(resp)

['劳动文化',
 '职工文化',
 '劳动艺术',
 '劳动教育',
 '职业精神',
 '企业文化',
 '员工培训',
 '职工福利',
 '职工文化',
 '劳动保障',
 '社会保障',
 '劳动保险',
 '工资保障',
 '劳动标准',
 '劳动关系']

In [15]:
t = [
    "人才引进政策",
    "引进人才的类别",
    "引进人才的程序",
    "引进人才的标准",
    "引进人才的待遇",
    "人才引进工作流程",
    "引进人才的渠道",
    "引进人才的管理",
    "人才引进服务"
]

In [38]:
query = '''中国人社体系中的劳动保障监察对象主要包含哪些内容，请将关键词放在一个Python的列表中作为答案返回'''
resp, his = model.chat(tokenizer=tokenizer, query=query)

In [39]:
print(resp)

中国人社体系中的劳动保障监察对象主要包括以下关键词：

1. 劳动保障
2. 劳动保障监察
3. 用人单位
4. 职工
5. 工资
6. 工时
7. 社会保险
8. 劳动争议
9. 职业介绍
10. 职业培训
11. 劳动保障法律法规
12. 劳动保障政策
13. 劳动保障待遇
14. 劳动保障权益
15. 劳动保障监管

将这些关键词放入一个Python列表中，可以表示为：

```python
关键词列表 = ['劳动保障', '劳动保障监察', '用人单位', '职工', '工资', '工时', '社会保险', '劳动争议', '职业介绍', '职业培训', '劳动保障法律法规', '劳动保障政策', '劳动保障待遇', '劳动保障权益', '劳动保障监管']
```


In [16]:
str(t).replace("'", '"')

'["人才引进政策", "引进人才的类别", "引进人才的程序", "引进人才的标准", "引进人才的待遇", "人才引进工作流程", "引进人才的渠道", "引进人才的管理", "人才引进服务"]'

In [14]:
print(resp)

中国人社体系中的人才引进主要包含以下内容：

```python
[
    "人才引进政策",
    "引进人才的类别",
    "引进人才的程序",
    "引进人才的标准",
    "引进人才的待遇",
    "人才引进工作流程",
    "引进人才的渠道",
    "引进人才的管理",
    "人才引进服务"
]
```


In [12]:
print(resp)

中国人社体系中的人才引进主要包含以下内容：

[1] 人才引进政策
[2] 人才引进计划
[3] 人才引进渠道
[4] 人才引进程序
[5] 人才引进标准
[6] 人才引进对象
[7] 人才引进地区
[8] 人才引进需求
[9] 人才引进效果评估
[10] 人才引进资金支持


In [64]:
import re

In [86]:
re.findall(r'```python\n([\s\S]*?)```', resp)[0].strip()

'human_resource_strategy_plan = [\n    "人力资源市场分析",\n    "人力资源需求预测",\n    "人力资源供给与需求平衡",\n    "人力资源政策与法规",\n    "人才培养与发展",\n    "人才引进与使用",\n    "人力资源信息与监测",\n    "社会保障体系",\n    "劳动争议处理",\n    "企业人力资源管理"\n]'

In [133]:
result = '[' + re.findall(r'\[([\s\S]*?)\]', resp)[0] + ']'

In [134]:
eval(result)

['人才引进政策',
 '人才引进计划',
 '人才引进程序',
 '人才引进标准',
 '人才引进对象',
 '人才引进渠道',
 '人才引进待遇',
 '人才引进评估',
 '人才引进服务']

In [90]:
eval(re.findall(r'\[([\s\S]*?)\]', resp)[0].strip())

IndentationError: unexpected indent (<string>, line 2)

In [66]:
resp

'中国人社体系中的人力资源战略规划主要包含以下内容：\n\n1. 人力资源市场分析\n2. 人力资源需求预测\n3. 人力资源供给与需求平衡\n4. 人力资源政策与法规\n5. 人才培养与发展\n6. 人才引进与使用\n7. 人力资源信息与监测\n8. 社会保障体系\n9. 劳动争议处理\n10. 企业人力资源管理\n\n这些关键词可以放入一个Python列表中，如下所示：\n\n```python\nhuman_resource_strategy_plan = [\n    "人力资源市场分析",\n    "人力资源需求预测",\n    "人力资源供给与需求平衡",\n    "人力资源政策与法规",\n    "人才培养与发展",\n    "人才引进与使用",\n    "人力资源信息与监测",\n    "社会保障体系",\n    "劳动争议处理",\n    "企业人力资源管理"\n]\n```'

In [2]:
t = [
    "人力资源市场分析",
    "人力资源需求预测",
    "人力资源供给与需求平衡",
    "人力资源政策与法规",
    "人才培养与发展",
    "人才引进与使用",
    "人力资源信息与监测",
    "社会保障体系",
    "劳动争议处理",
    "企业人力资源管理"
]

In [5]:
str(t).replace("'", '"')

'["人力资源市场分析", "人力资源需求预测", "人力资源供给与需求平衡", "人力资源政策与法规", "人才培养与发展", "人才引进与使用", "人力资源信息与监测", "社会保障体系", "劳动争议处理", "企业人力资源管理"]'

In [63]:
print(resp)

中国人社体系中的人力资源战略规划主要包含以下内容：

1. 人力资源市场分析
2. 人力资源需求预测
3. 人力资源供给与需求平衡
4. 人力资源政策与法规
5. 人才培养与发展
6. 人才引进与使用
7. 人力资源信息与监测
8. 社会保障体系
9. 劳动争议处理
10. 企业人力资源管理

这些关键词可以放入一个Python列表中，如下所示：

```python
human_resource_strategy_plan = [
    "人力资源市场分析",
    "人力资源需求预测",
    "人力资源供给与需求平衡",
    "人力资源政策与法规",
    "人才培养与发展",
    "人才引进与使用",
    "人力资源信息与监测",
    "社会保障体系",
    "劳动争议处理",
    "企业人力资源管理"
]
```


In [ ]:
office_duties = [
    "政务综合协调", "文件运转", "文件管理", "会务组织", "部值班", "文电处理", "信息管理", "督查", "机要工作",
    "档案管理", "机关财务", "安全保密", "信访维稳", "文印工作", "人大代表建议提案", "全国政协委员提案", "政务公开",
    "门户网站管理", "办公内网管理", "领导交办事项"
]

In [29]:
print(resp)

办公厅职责的关键词可以总结为：综合协调、文件运转、会议会务、值班处理、信息督查、机要档案、财务安全、信访稳定、文印、建议提案、政务公开、门户网站、办公内网、领导交办。

对应的python列表形式为：

[
    "综合协调",
    "文件运转",
    "会议会务",
    "值班处理",
    "信息督查",
    "机要档案",
    "财务安全",
    "信访稳定",
    "文印",
    "建议提案",
    "政务公开",
    "门户网站",
    "办公内网",
    "领导交办"
]


In [8]:
dict(df.iloc[:, [0, 1]].values)

{'rsorgan1': '办公厅',
 'rsorgan2': '政策研究司',
 'rsorgan3': '法规司',
 'rsorgan4': '规划财务司',
 'rsorgan5': '就业促进司',
 'rsorgan6': '人力资源流动管理司',
 'rsorgan7': '职业能力建设司',
 'rsorgan8': '专业技术人员管理司',
 'rsorgan9': '事业单位人事管理司',
 'rsorgan10': '农民工工作司',
 'rsorgan11': '劳动关系司',
 'rsorgan12': '工资福利司',
 'rsorgan13': '养老保险司',
 'rsorgan14': '失业保险司',
 'rsorgan15': '工伤保险司',
 'rsorgan16': '农村社会保险司',
 'rsorgan17': '社会保险基金监管局',
 'rsorgan18': '调解仲裁管理司',
 'rsorgan19': '劳动保障监察局',
 'rsorgan20': '国家表彰奖励办公室',
 'rsorgan21': '国际合作司（港澳台办公室）',
 'rsorgan22': '人事司'}

In [20]:
departments_nodes = []
for index, row in df.iterrows():
    departments_nodes.append({
        'rsorgan_Id': row['rsorgan_Id:ID'],
        'name': row['name']
    })

In [22]:
create_node('人社部内设机构', departments_nodes)

creating 人社部内设机构 nodes


100%|██████████| 22/22 [00:00<00:00, 53.89it/s]


In [19]:
df

,rsorgan_Id:ID,name,:LABEL
0,rsorgan1,办公厅,人社部内设机构
1,rsorgan2,政策研究司,人社部内设机构
2,rsorgan3,法规司,人社部内设机构
3,rsorgan4,规划财务司,人社部内设机构
4,rsorgan5,就业促进司,人社部内设机构
5,rsorgan6,人力资源流动管理司,人社部内设机构
6,rsorgan7,职业能力建设司,人社部内设机构
7,rsorgan8,专业技术人员管理司,人社部内设机构
8,rsorgan9,事业单位人事管理司,人社部内设机构
9,rsorgan10,农民工工作司,人社部内设机构
